# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_file = "../Resources/city_weather_data.csv"

# Read our Kickstarter data into pandas
df = pd.read_csv(city_data_file)
df.head()

,Unnamed: 0,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,saint-philippe,75.13,-21.3585,55.7679,75.20,73,0,15.12,RE,1619840582
1,1,bambous virieux,77.47,-20.3428,57.7575,78.01,78,40,10.36,MU,1619840922
2,2,saldanha,48.20,-33.0117,17.9442,48.20,81,0,2.30,ZA,1619840190
3,3,sfantu gheorghe,47.98,45.8667,25.7833,48.99,58,78,1.70,RO,1619840923
4,4,casablanca,48.15,33.5928,-7.6192,50.00,100,75,3.44,MA,1619840670


In [18]:
city_weather_data_df = df.drop(columns=['Unnamed: 0'])
city_weather_data_df.head()

,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,saint-philippe,75.13,-21.3585,55.7679,75.20,73,0,15.12,RE,1619840582
1,bambous virieux,77.47,-20.3428,57.7575,78.01,78,40,10.36,MU,1619840922
2,saldanha,48.20,-33.0117,17.9442,48.20,81,0,2.30,ZA,1619840190
3,sfantu gheorghe,47.98,45.8667,25.7833,48.99,58,78,1.70,RO,1619840923
4,casablanca,48.15,33.5928,-7.6192,50.00,100,75,3.44,MA,1619840670


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure()

In [20]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_data_df[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
# census_data_complete = census_data_complete.dropna()
humidity_rate = city_weather_data_df["Humidity"].astype(float)

In [21]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Filter down by 3 conditions for ideal weather
filtered_df = city_weather_data_df.loc[(city_weather_data_df['Max Temp']>=70) & (city_weather_data_df['Max Temp']< 80) & (city_weather_data_df['Wind'] < 10)]

In [25]:
# check to see if there are any NA
filtered_df.count()

City Name       75
Current Temp    75
Latitude        75
Longitude       75
Max Temp        75
Humidity        75
Cloudiness      75
Wind            75
Country         75
Date            75
dtype: int64

In [26]:
# run if there are NaN to eliminate
filtered_df.dropna(how='all')

,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date
14,san patricio,73.85,28.0170,-97.5169,75.20,94,90,8.05,US,1619840928
28,hobart,67.51,-42.8794,147.3294,71.60,35,75,9.22,AU,1619840524
36,grand gaube,78.01,-20.0064,57.6608,78.01,83,100,4.00,MU,1619840935
43,alofi,77.00,-19.0595,-169.9187,77.00,94,100,2.30,NU,1619840583
44,cabo san lucas,71.69,22.8909,-109.9124,73.00,68,0,1.99,MX,1619840694
...,...,...,...,...,...,...,...,...,...,...
551,asyut,73.40,27.1810,31.1837,73.40,35,0,9.22,EG,1619841243
553,elko,69.46,41.0002,-115.5012,70.00,19,100,5.01,US,1619840961
556,sakakah,75.20,29.9697,40.2064,75.20,17,0,9.22,SA,1619841251
562,gulfport,70.07,30.3674,-89.0928,71.60,78,1,5.75,US,1619841263


In [27]:
filtered_df.count()

City Name       75
Current Temp    75
Latitude        75
Longitude       75
Max Temp        75
Humidity        75
Cloudiness      75
Wind            75
Country         75
Date            75
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# create new hotel data frame and add new column for hotel name
hotel_df = filtered_df

hotel_df['Hotel Name'] = ""

hotel_df.head(10)

<ipython-input-31-c3b73b899dda>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
14,san patricio,73.85,28.0170,-97.5169,75.20,94,90,8.05,US,1619840928,
28,hobart,67.51,-42.8794,147.3294,71.60,35,75,9.22,AU,1619840524,
36,grand gaube,78.01,-20.0064,57.6608,78.01,83,100,4.00,MU,1619840935,
43,alofi,77.00,-19.0595,-169.9187,77.00,94,100,2.30,NU,1619840583,
44,cabo san lucas,71.69,22.8909,-109.9124,73.00,68,0,1.99,MX,1619840694,
48,hasaki,73.85,35.7333,140.8333,75.00,43,75,9.22,JP,1619840596,
50,vila velha,73.92,-20.3297,-40.2925,75.99,73,75,8.05,BR,1619840883,
61,avarua,78.80,-21.2078,-159.7750,78.80,73,20,5.75,CK,1619840558,
65,road town,77.67,18.4167,-64.6167,78.01,82,0,3.00,VG,1619840947,
78,cayenne,76.23,4.9333,-52.3333,77.00,94,90,6.17,GF,1619840311,


In [37]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}


print("Begin Hotel Search by Location")
print("------------------------------")

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

print("-------------------------------------")
print("Nearest Hotel Search is now complete.")

------------------------------
Begin Hotel Search by Location


/Users/jennamurphy/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
-------------------------------------
Nearest Hotel Search is now complete.


In [38]:
hotel_df.head()

,City Name,Current Temp,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
14,san patricio,73.85,28.0170,-97.5169,75.20,94,90,8.05,US,1619840928,Americas Best Value Inn Sinton
28,hobart,67.51,-42.8794,147.3294,71.60,35,75,9.22,AU,1619840524,Hotel Grand Chancellor Hobart
36,grand gaube,78.01,-20.0064,57.6608,78.01,83,100,4.00,MU,1619840935,LUX Grand Gaube Resort & Villas
43,alofi,77.00,-19.0595,-169.9187,77.00,94,100,2.30,NU,1619840583,Taloa Heights
44,cabo san lucas,71.69,22.8909,-109.9124,73.00,68,0,1.99,MX,1619840694,Bahia Hotel & Beach House


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
humidity_rate_2 = hotel_df["Humidity"].astype(float)
fig_2 = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_layer_2 = gmaps.heatmap_layer(locations, weights=humidity_rate_2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig_2.add_layer(heat_layer_2)
fig_2.add_layer(marker_layer)
fig_2

Figure(layout=FigureLayout(height='420px'))